In [1]:
import pandas as pd
# 범주형 데이터 벡터화
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train = pd.read_csv('../../main/data/origin/train.csv')
df_train['군구'] = df_train['시군구'].replace('대구광역시 ','',regex=True)
df_train['군구'] = le.fit_transform(df_train['군구'])
df_train['사고일시'] = pd.to_datetime(df_train['사고일시'])
df_train['월'] = df_train['사고일시'].dt.month
df_train['시간'] = df_train['사고일시'].dt.hour
df_train.drop(columns=['ID','시군구','사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도',
       '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도','사고일시','사망자수', '중상자수', '경상자수', '부상자수'],inplace=True)

In [2]:
df_train = pd.get_dummies(df_train, columns = ['요일', '기상상태', '도로형태', '노면상태', '사고유형'])

In [3]:
X_trn = df_train.drop(columns='ECLO')
y_trn = df_train['ECLO']

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'cuda'

In [5]:
from tqdm.auto import tqdm
from typing import Optional, List
import torchmetrics
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error as mse

c:\users\zpdlc\appdata\local\programs\python\python311\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class CustomDataset(Dataset):
  def __init__(self, *args:list[np.array]):
    assert all(args[0].shape[0] == arg.shape[0] for arg in args), "Size mismatch."
    self.data = args
  def __getitem__(self, index):
    return tuple(x[index] for x in self.data)
  def __len__(self):
    return self.data[0].shape[0]

In [7]:
X_trn

,군구,월,시간,요일_금요일,요일_목요일,요일_수요일,요일_월요일,요일_일요일,요일_토요일,요일_화요일,...,도로형태_주차장 - 주차장,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,사고유형_차대사람,사고유형_차대차,사고유형_차량단독
0,155,1,0,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,True,False,False
1,4,1,0,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,True,False,False
2,124,1,1,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,True,False,False
3,98,1,2,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False
4,65,1,4,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,135,12,19,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
39605,11,12,19,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
39606,16,12,21,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
39607,21,12,22,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False


In [14]:
import torch.nn as nn
import torch
import pandas as pd

activation_list = {"sigmoid": nn.Sigmoid(), "relu": nn.ReLU(), "tanh": nn.Tanh(), "prelu": nn.PReLU()}

class ANN(nn.Module):
  def __init__(self, input_dim: int=5, hidden_dim: list=[128, 128, 64, 32], activation: str="sigmoid", use_drop:bool = True, drop_ratio: float=0.3):
    super().__init__()
    self.embedding = nn.Linear(1,10) # '군구'피쳐를 1차원(노드)에서 10차원(노드) 으로 변경
    dims = [input_dim] + hidden_dim 
    self.Identity = nn.Identity()
    self.dropout = nn.Dropout(drop_ratio)
    self.activation = activation_list[activation]
    
    model = [[nn.Linear(dims[i], dims[i+1]), self.dropout if use_drop else self.Identity, self.activation] for i in range(len(dims) - 1)]
    output_layer = [nn.Linear(dims[-1], 1), nn.Identity()] # 1 -> 4 하고 1 -> 사망자, data unvalance가 있어도 괜찮  mse를 활용하며 가중치를 사망자, 경상자 마다 바꾸고 학습
    # output_layer = [nn.Linear(dims[-1], 1), nn.Identity()] # Relu는 항상 양수 값일 때 주는 방법
    self.module_list= nn.ModuleList(sum(model, []) + output_layer)
  def forward(self, x):
    x = torch.concat([x[:,1:], self.embedding(x[:,0:1])],dim=1) # self.embedding(x): '군구' x: 나머지
    for layer in self.module_list:
         x = layer(x) # 차원 맞춰서 갯수증가(채용) # 다른방법: linear layer하나 추가하고 다른 피쳐들 가중치를 0 으로 학습 진행 x
    return x

In [9]:

from torch.nn import MSELoss
def train_one_epoch(
  model:nn.Module,
  criterion:callable,
  optimizer:torch.optim.Optimizer,
  data_loader:DataLoader,
  device:str
) -> float:
  '''train one epoch

  Args:
      model: model
      criterion: loss
      optimizer: optimizer
      data_loader: data loader
      device: device
  '''
  model.train()
  total_loss = 0.
  for X, y in data_loader:
    X, y = X.to(device), y.to(device)
    output = model(X)
    loss = criterion(output.squeeze(-1), y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item() * len(y)
  return total_loss/len(data_loader.dataset)

def evaluate(
  model: nn.Module,
  criterion: callable,
  data_loader: DataLoader,
  device: str,
  metric: Optional[torchmetrics.metric.Metric] = None,
  multi_metrics: List[torchmetrics.metric.Metric] = None
) -> float:
  '''evaluate

  Args:
      model: model
      criterions: list of criterion functions
      data_loader: data loader
      device: device
  '''
  model.eval()
  total_loss = 0.
  with torch.no_grad():  # no_grad context로 감싸서 그라디언트 계산 비활성화
    for X, y in data_loader:
      X, y = X.to(device), y.to(device)
      output = model(X)
      total_loss += criterion(output.squeeze(-1), y).item() * len(y)

      if metric is not None:
        # For a single metric, update it with the total output and total target
        metric.update(output.squeeze(-1), y)

      if multi_metrics is not None:
        # For multiple metrics, update each metric with the current output and target
        for metric in multi_metrics:
          metric.update(output.squeeze(-1), y)

    # 평가 지표 계산 시에는 스칼라인 경우에 대한 처리 추가
    if isinstance(total_loss, torch.Tensor): # isinstance(x,y) x의 타입이 y가 맞는지 T,F
        return total_loss.item() / len(data_loader.dataset)
    else:
        return total_loss / len(data_loader.dataset)


def kfold_cross_validation(model: nn.Module, criterion:callable, device:str, X_trn:np.array, y_trn:np.array, n_splits:int=5):
  from sklearn.model_selection import KFold
  from torchmetrics import MeanAbsoluteError, MeanSquaredError, MeanSquaredLogError
  # from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_squared_log_error
  from copy import deepcopy
  
  Kf = KFold(n_splits=n_splits, shuffle=True, random_state=2023)
  nets = [deepcopy(model) for i in range(n_splits)]
  scores = {
  'mae': [],
  'mse': [],
  'msle': []
  }
  
  for i, (trn_idx, val_idx) in enumerate(Kf.split(X_trn, y_trn)):
    X, y = torch.tensor(X_trn.iloc[trn_idx].values.astype(np.float32)), torch.tensor(y_trn.iloc[trn_idx].values.astype(np.float32))
    X_val, y_val = torch.tensor(X_trn.iloc[val_idx].values.astype(np.float32)), torch.tensor(y_trn.iloc[val_idx].values.astype(np.float32))

    ds = CustomDataset(X, y)
    ds_val = CustomDataset(X_val, y_val)
    dl = DataLoader(ds, batch_size=64, shuffle=True)
    dl_val = DataLoader(ds_val, batch_size=len(ds_val), shuffle=False)

    net = nets[i].train()

    pbar = tqdm(range(30))
    for j in pbar:
      mae, mse, msle = MeanAbsoluteError().to(device), MeanSquaredError().to(device), MeanSquaredLogError().to(device)
      optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
      loss = train_one_epoch(net, criterion, optimizer, dl, device)
      loss_val = evaluate(net, criterion, dl_val, device, multi_metrics=[mae, mse, msle])
      mae, mse, msle = mae.compute(), mse.compute(), msle.compute()
      # Get predictions for validation set
      # y_val_pred = net(X_val).detach().numpy()
      pbar.set_postfix(trn_loss=loss, val_loss=loss_val)  # 진행바 우측에 진행상황 표시
    scores["mae"].append(mae.item())
    scores["mse"].append(mse.item())
    scores["msle"].append(msle.item())

  return scores

In [15]:
device = torch.device(device)
X_trn = df_train.drop(columns='ECLO')
y_trn = df_train['ECLO']

ds = CustomDataset(X_trn, y_trn)
dl = DataLoader(ds, batch_size=64, shuffle=True)

model = ANN(X_trn.shape[-1], hidden_dim=[64,64], drop_ratio = 0.3).to(device)
print(model)
loss_func = nn.functional.mse_loss

scores = kfold_cross_validation(model, loss_func, device, X_trn, y_trn)
scores_df = pd.DataFrame(scores)
scores_df = pd.concat([scores_df, scores_df.apply(['mean', 'std'])])
scores_df.to_csv("scores.csv", index=False)
print(scores_df)


ANN(
  (Identity): Identity()
  (dropout): Dropout(p=0.3, inplace=False)
  (activation): Sigmoid()
  (module_list): ModuleList(
    (0): Linear(in_features=36, out_features=64, bias=True)
    (1): Dropout(p=0.3, inplace=False)
    (2): Sigmoid()
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): Dropout(p=0.3, inplace=False)
    (5): Sigmoid()
    (6): Linear(in_features=64, out_features=1, bias=True)
    (7): Identity()
  )
)


100%|██████████| 30/30 [00:48<00:00,  1.61s/it, trn_loss=10.1, val_loss=9.84]

           mae        mse      msle
0     2.172371  10.602310  0.212433
1     2.108865   9.134976  0.207503
2     2.114625  10.476859  0.207939
3     2.177469  10.178031  0.216804
4     2.125452   9.838398  0.212150
mean  2.139756  10.046115  0.211366
std   0.032697   0.588561  0.003807
